In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

%matplotlib inline

In [ ]:
df_in = pd.read_csv('Austin_Animal_Center_Intakes.csv')
df_in.head()

# Preparing the data

In [ ]:
df_out = pd.read_csv('Austin_Animal_Center_Outcomes.csv')
df_out.head()

In [ ]:
# Join the intake data with the outcome data;
# include date and location it was found, intake type and condition,
# and whether it was neutered/spayed or not at intake

df_in1 = df_in[['Animal ID', 'Name', 'DateTime', 'MonthYear', 'Found Location', 'Intake Type', 'Intake Condition', 'Animal Type', 'Sex upon Intake', 'Age upon Intake', 'Breed', 'Color']].rename(columns={'DateTime': 'Intake Time', 'MonthYear' : 'Intake MonthYear'})

df_out1 = df_out[['Outcome Type', 'DateTime', 'MonthYear', 'Outcome Subtype', 'Sex upon Outcome', 'Age upon Outcome']].rename(columns={'DateTime': 'Outcome Time', 'MonthYear': 'Outcome MonthYear'})

df_combined = pd.concat([df_in1, df_out1])


print(df_combined)

In [ ]:
# TODO 2: find the unique type-subtype pairs. Right now it just shows them all
type_subtype_pairing = df_out[['Outcome Type', 'Outcome Subtype']]
outcome_type = df_out['Outcome Type']
outcome_subtype = df_out['Outcome Subtype']

unique_types = outcome_type.unique()
unique_subtypes = outcome_subtype.unique()

# Unique type-subtype pairings, such as 'Adoption' and 'Foster'
unique_pairings = []

print("Unique types of outcomes - ", unique_types.tolist())
print("Unique sub-types of outcomes - ", unique_subtypes.tolist())

total_cases = 0
for outcomeType in unique_types:
    outcome_count = len(type_subtype_pairing.loc[type_subtype_pairing['Outcome Type'] == outcomeType])
    total_cases += outcome_count
    print('Number of cases where the outcome is {} is {}'.format(outcomeType, outcome_count))

print('The total number of cases is', total_cases)

print('\n\nType-Subtype pairing -\n', type_subtype_pairing)

In [ ]:
df_out = df_out.convert_dtypes(infer_objects=True)
df_out['DateTime'] = pd.to_datetime(df_out['DateTime'])
df_out['Date of Birth'] = pd.to_datetime(df_out['Date of Birth'])
df_out.info()

## Name

In [ ]:
# Set df_out['Name'] so that if it had an asterisk in front, the asterisk is removed

nameNA = df_out['Name'].isna()
print(len(nameNA), len(df_out['Name']))


df_out['Name'] = [name.replace('*', '') if nameNA[i] == False else name for i, name in enumerate(df_out['Name'])]

df_out.head()

## Colors

In [ ]:
from math import pi

# colors.csv was compiled from these wikipedia articles
# https://en.wikipedia.org/wiki/List_of_colors:_A–F
# https://en.wikipedia.org/wiki/List_of_colors:_G%E2%80%93M
# https://en.wikipedia.org/wiki/List_of_colors:_N%E2%80%93Z
# Then the "—" character was replaced with "0"
df_colors = pd.read_csv('colors.csv')
df_colors = df_colors.convert_dtypes(infer_objects=True)
df_colors['Name'] = df_colors['Name'].str.lower()
df_colors['Red (RGB)'] = pd.to_numeric(df_colors['Red (RGB)'].str.replace('%', '')).div(100)
df_colors['Green (RGB)'] = pd.to_numeric(df_colors['Green (RGB)'].str.replace('%', '')).div(100)
df_colors['Blue (RGB)'] = pd.to_numeric(df_colors['Blue (RGB)'].str.replace('%', '')).div(100)
df_colors['Hue (HSL/HSV)'] = pd.to_numeric(df_colors['Hue (HSL/HSV)'].str.replace('°', '')).div(360)
df_colors['Satur. (HSL)'] = pd.to_numeric(df_colors['Satur. (HSL)'].str.replace('%', '')).div(100)
df_colors['Light (HSL)'] = pd.to_numeric(df_colors['Light (HSL)'].str.replace('%', '')).div(100)
df_colors['Satur. (HSV)'] = pd.to_numeric(df_colors['Satur. (HSV)'].str.replace('%', '')).div(100)
df_colors['Value (HSV)'] = pd.to_numeric(df_colors['Value (HSV)'].str.replace('%', '')).div(100)
df_colors.head()

Since we will look for correlations with the color variables for the animals, we want to account for the fact that those color variables are drawn from this dataset, and so there could be a sampling bias in how the colors are interpreted.

Here, it looks like there is a weak correlation between red and green and between green and blue, but red and blue are mostly unrelated.

In [ ]:
sns.heatmap(data=df_colors.corr(), annot=True)

In [ ]:
sns.pairplot(df_colors)

Colors are looked up by splitting the query color name by words, then looking for the first exact match with one of the words, but if there are none then it looks for any color with a name containing any of the query words.

For example, consider looking up colors with the word "brown" in their name. Notice that none of the colors are literally "brown" though they all have "brown" in the name.

In [ ]:
df_colors.loc[df_colors.Name.str.contains("brown")]

In [ ]:
def colorInfo(color):
    color = color.lower()
    words = [color] if color.count(' ') == 0 else [color] + color.split(' ')
    for word in words:
        try:
            items = df_colors.loc[df_colors.Name == word]
            if len(items) > 0:
                return items
        except:
            continue
    
    for word in words:
        try:
            items = df_colors.loc[df_colors.Name.str.contains(word)]
            if len(items) > 0:
                return items
        except:
            continue
        
    return None

# def rgb(color):
#     info = colorInfo(color)
#     if info is None: return (None, None, None)
#     r = info['Red (RGB)'].values[0]
#     g = info['Green (RGB)'].values[0]
#     b = info['Blue (RGB)'].values[0]
#     return (r, g, b)

def rgbhsv(color):
    info = colorInfo(color)
    if info is None: return (None, None, None, None, None, None)
    r = info['Red (RGB)'].values[0]
    g = info['Green (RGB)'].values[0]
    b = info['Blue (RGB)'].values[0]
    h = info['Hue (HSL/HSV)'].values[0]
    s = info['Satur. (HSV)'].values[0]
    v = info['Value (HSV)'].values[0]
    return (r, g, b, h, s, v)

rgbhsv('brown')

In [ ]:
def flatten(x):
    res = []
    for y in x:
        res.extend(y)
    return res

flatten([(1, 2), ('x', 'y')])

In [ ]:
df_out['Colors (count)'] = df_out.Color.str.count('/') + 1
df_out = df_out.assign(**{
    'Color 0': [colors[0] for colors in df_out.Color.str.split('/')],
    'Color 1': [colors[1] if len(colors) > 1 else None for colors in df_out.Color.str.split('/')]
}).convert_dtypes(infer_objects=True)
df_out.info()
df_out.head()

This cell takes a few minutes to complete

In [ ]:
# TODO 4 : Find some way to optimize this

color_vars = 'RGBHSV'

for color_index in ['0', '1']:
    colors = df_out[f'Color {color_index}']
    colors_isna = colors.isna()
    colors_RGBHSV = [rgbhsv(color) if not colors_isna[i] else (None, None, None, None, None, None) for i,color in enumerate(colors)]
    color_columns = []
    for color_var in color_vars:
        color_column = f'Color {color_index} {color_var}'
        color_columns.append(color_column)
        df_out = df_out.assign(**{
            color_column: pd.Series(np.zeros_like(df_out.index)).astype(dtype=float)
        })
    df_out[color_columns] = colors_RGBHSV

df_out

## Age

The "Age upon Outcome (years)" column is made here

In [ ]:
def age_years(age):
    try:
        [number_str, unit] = age.split(' ')
        number = float(number_str)
        if unit in ['years', 'year']:
            return number
        elif unit in ['months', 'month']:
            return number / 12
        elif unit in ['weeks', 'week']:
            return number / 52
        elif unit in ['days', 'day']:
            return number / 365
    except: pass
    return None

df_out['Age upon Outcome (years)'] = [age_years(age) for age in df_out['Age upon Outcome']]
df_out.head()

## Sex

Male or female are classified in two columns since some animals are of unknown sex

In [37]:
# sex and neuter/spay columns

sex_male = df_out['Sex upon Outcome'].str.endswith(' Male')
sex_female = df_out['Sex upon Outcome'].str.endswith(' Female')
neutered_or_spayed = df_out['Sex upon Outcome'].str.startswith('Neutered') | df_out['Sex upon Outcome'].str.startswith('Spayed')

df_out['Male'] = sex_male
df_out['Female'] = sex_female
df_out['NeuteredOrSpayed'] = neutered_or_spayed
df_out = df_out.convert_dtypes(convert_boolean=True)
df_out.info()
df_out.head(10)

0          True
1          True
2         False
3          True
4         False
          ...  
149506     True
149507     True
149508     True
149509     True
149510     True
Name: Sex upon Outcome, Length: 149511, dtype: boolean
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149511 entries, 0 to 149510
Data columns (total 32 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   Animal ID                 149511 non-null  string        
 1   Name                      106260 non-null  string        
 2   DateTime                  149511 non-null  datetime64[ns]
 3   MonthYear                 149511 non-null  string        
 4   Date of Birth             149511 non-null  datetime64[ns]
 5   Outcome Type              149485 non-null  string        
 6   Outcome Subtype           68443 non-null   string        
 7   Animal Type               149511 non-null  string        
 8   Sex upon Outcome       

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,...,Color 1 G,Color 1 B,Color 1 H,Color 1 S,Color 1 V,Age upon Outcome (years),Male,Female,NeuteredOrSpayed,Adopted
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,<NA>,Cat,Neutered Male,2 years,...,1.0,1.0,0.0,0.0,1.0,2.0,True,False,True,True
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,<NA>,Dog,Neutered Male,1 year,...,0.44,0.09,0.119444,0.85,0.59,1.0,True,False,True,True
2,A821648,<NA>,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,<NA>,Other,Unknown,1 year,...,<NA>,<NA>,<NA>,<NA>,<NA>,1.0,False,False,False,False
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,<NA>,Dog,Neutered Male,4 months,...,<NA>,<NA>,<NA>,<NA>,<NA>,0.333333,True,False,True,True
4,A674754,<NA>,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,...,<NA>,<NA>,<NA>,<NA>,<NA>,0.016438,True,False,False,False
5,A659412,Princess,2020-10-05 14:37:00,Oct 2020,2013-03-24,Adoption,<NA>,Dog,Spayed Female,7 years,...,<NA>,<NA>,<NA>,<NA>,<NA>,7.0,False,True,True,True
6,A814515,Quentin,2020-05-06 07:59:00,May 2020,2018-03-01,Adoption,Foster,Dog,Neutered Male,2 years,...,0.44,0.09,0.119444,0.85,0.59,2.0,True,False,True,True
7,A868405,Leo,2023-03-04 13:38:00,Mar 2023,2020-11-02,Adoption,<NA>,Dog,Neutered Male,2 years,...,1.0,1.0,0.0,0.0,1.0,2.0,True,False,True,True
8,A689724,Donatello,2014-10-18 18:52:00,Oct 2014,2014-08-01,Adoption,<NA>,Cat,Neutered Male,2 months,...,<NA>,<NA>,<NA>,<NA>,<NA>,0.166667,True,False,True,True
9,A680969,Zeus,2014-08-05 16:59:00,Aug 2014,2014-06-03,Adoption,<NA>,Cat,Neutered Male,2 months,...,0.5,0.0,0.083333,1.0,1.0,0.166667,True,False,True,True


## Adopted?

In [ ]:
df_out['Adopted'] = df_out['Outcome Type'].str.find('Adopt') >= 0

In [ ]:
def bigCorr_bernoulli(df, independent, dependent):
    return (
        df[[independent, dependent]].groupby(independent).value_counts().div(
        df[[independent]].groupby(independent).value_counts())
    )[:,True]

## Preparing the breeds dataframe

In [ ]:
breeds_counts = df_out.Breed.value_counts()
df_breeds = pd.DataFrame(index=breeds_counts.index)
df_breeds = df_breeds.assign(Breed=breeds_counts.index, Count=breeds_counts)
df_breeds

In [ ]:
# Add animal type to the breed
breeds_types = df_out[['Breed', 'Animal Type']].groupby('Breed').value_counts().index
breeds = [breed_type[0] for breed_type in breeds_types]
types = [breed_type[1] for breed_type in breeds_types]
df_breeds_types = pd.DataFrame(data=types, index=breeds, columns=['Type'])
df_breeds_types = df_breeds_types.assign(Breed=breeds)
df_breeds_types = df_breeds_types.drop_duplicates('Breed')
df_breeds = df_breeds.assign(**{'Animal Type': df_breeds_types['Type']})
df_breeds = df_breeds.convert_dtypes(infer_objects=True)
df_breeds

In [ ]:
## Adoption likelihood

df_breeds = df_breeds.assign(Adopted=bigCorr_bernoulli(df_out, 'Breed', 'Adopted'))
df_breeds

In [ ]:
def means(df_groups, df_individual, independent, dependent):
    df_groups[f'{dependent} (mean)'] = df_individual[[independent, dependent]].groupby(independent).mean()
def stdDevs(df_groups, df_individual, independent, dependent):
    df_groups[f'{dependent} (std dev)'] = df_individual[[independent, dependent]].groupby(independent).std()
def stats(df_groups, df_individual, independent, dependent):
    means(df_groups, df_individual, independent, dependent)
    stdDevs(df_groups, df_individual, independent, dependent)

for color_index in ['0', '1']:
    for color_feature in color_vars:
        stats(df_breeds, df_out, 'Breed', f'Color {color_index} {color_feature}')

df_breeds.head()

In [ ]:
# sns.pairplot(data=df_breeds)

In [ ]:
df_breeds_info = pd.read_csv('dog breeds_enriched_20210503.csv').convert_dtypes(infer_objects=True)
df_breeds_info.info()
df_breeds_info.head()

In [ ]:
sns.pairplot(data=df_breeds_info)

In [ ]:
breeds_names_lower = df_breeds_info.Breed.str.lower()

def findBreedInfoName(breed):
    breed = breed.lower()
    
    info = df_breeds_info.Breed.loc[breeds_names_lower.str.contains(breed)]
    if len(info) > 0: return info.values[0]
    
    for word in breed.split(' '):
        info = df_breeds_info.Breed.loc[breeds_names_lower.str.contains(word)]
        if len(info) > 0: return info.values[0]
    
    return None

In [ ]:
df_out_with_breeds_info = df_out.assign(BreedsInfoName=[findBreedInfoName(breed) for breed in df_out.Breed])
df_out_with_breeds_info = df_out_with_breeds_info.merge(df_breeds_info, how='left', left_on='BreedsInfoName', right_on='Breed')
df_out_with_breeds_info.rename(columns={'Breed_x': 'Breed', 'Breed_y': 'Breed (catalog)'}, inplace=True)
df_out_with_breeds_info.info()
df_out_with_breeds_info.head()

In [ ]:
df_breeds_with_info = df_breeds.assign(BreedsInfoName=[findBreedInfoName(breed) for breed in df_breeds.Breed])
df_breeds_with_info = df_breeds_with_info.merge(df_breeds_info, how='left', left_on='BreedsInfoName', right_on='Breed')
df_breeds_with_info.rename(columns={'Breed_x': 'Breed', 'Breed_y': 'Breed (catalog)'}, inplace=True)
df_breeds_with_info.info()
df_breeds_with_info.head()

In [ ]:
df_out_with_breeds_info['Est. lifespan remaining'] = df_out_with_breeds_info['average lifespan'] - df_out_with_breeds_info['Age upon Outcome (years)']
df_out_with_breeds_info['Est. lifespan remaining'] = df_out_with_breeds_info['Est. lifespan remaining'].astype(dtype=float)
df_out_with_breeds_info.info()
df_out_with_breeds_info['Est. lifespan remaining']

In [ ]:
df_out.to_pickle('df_out.pkl')
df_breeds.to_pickle('df_breeds.pkl')
df_out_with_breeds_info.to_pickle('df_out_with_breeds_info.pkl')
df_breeds_with_info.to_pickle('df_breeds_with_info.pkl')

Data frames can now be quickly loaded for analysis

In [ ]:
df_out = pd.read_pickle('df_out.pkl')
df_breeds = pd.read_pickle('df_breeds.pkl')
df_out_with_breeds_info = pd.read_pickle('df_out_with_breeds_info.pkl')
df_breeds_with_info = pd.read_pickle('df_breeds_with_info.pkl')
df_out.info()
df_out.head()